In [1]:
import arcgis as GIS
import arcpy.mp
import os
import arcpy
import string, random

## New Functions June 2021
listWorkSpaces(workPath), listGDBitems(GDBpath), deleteFeatureClass(GDBpath,fcName), newFCfromSelect(GDBpath, selectFrom, selector, selType)

In [38]:
# Return list of workspaces in specified arcpy.env.workspace path
def listWorkSpaces(workPath):
    try:
        arcpy.env.workspace = workPath
        wsList = []
        ws = arcpy.ListWorkspaces("*", "FileGDB")
        for i,w in enumerate(ws):
            wsList.append(w)
            print(f'{i} - {w}')
        return wsList
    except:
        display(f'ERROR listing WorkSpaces in {workPath}')
        
# function to list all Feature Classes in specified FileGeoDatabase
def listItems(GDBpath):
    try:
        arcpy.env.workspace = GDBpath
        fcList = []
        fc = arcpy.ListFeatureClasses()
        print(f'Feature Classes/Shapefile in {GDBpath}: ')
        for i,f in enumerate(fc):
            fcList.append(f)
            print(f'{i} - {f}')
        return fcList
    except:
        display(f'ERROR listing Feature Classes in {GDBpath}')

# Delete feature class given path to GDB and fc name
def deleteFeatureClass(GDBpath,fcName):
    try:
        quest = (input(f'DELETE FEATURE CLASS - {GDBpath}\\{fcName} ??? - ') or 'NO').upper()
        if quest == 'YES':
            result = arcpy.Delete_management(f'{GDBpath}\\{fcName}')
            display(f'File: {fcName} delete = {result}')
        else:
            display(f'Did not delete: {fcName}')
    except:
        display(f'ERROR DELETING {fcName}')


# Make Selecttion and Create New Feature Class into function
def newFCfromSelect(GDBpath, selectFrom, selector, selType):
    try:
        switchSel = (input(f'Switch the resulting selection(y/N)? ')).upper()
        import string, random
        # set workspace GDB, overwrite = True
        arcpy.env.workspace,arcpy.env.overwriteOutput = GDBpath, True
        arcpy.Compact_management(GDBpath)
        # create random string to append to file name, prevents duplication errors
        rand = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 8))
        # tempLayer name from selectFrom name and random 8 character string
        # clear any previous selection and create temporary layer
        arcpy.SelectLayerByAttribute_management(selectFrom, "CLEAR_SELECTION")
        arcpy.MakeFeatureLayer_management(selectFrom, 'tempLayer') 
        # clear selection on selector and perfrom on temporary layer
        arcpy.SelectLayerByAttribute_management('tempLayer', "CLEAR_SELECTION")
        arcpy.SelectLayerByLocation_management('tempLayer', selType, selector)
        if switchSel == 'Y':
            arcpy.SelectLayerByAttribute_management('tempLayer', "SWITCH_SELECTION")
        # Check to see if any features were selected
        matchcount = int(arcpy.GetCount_management('tempLayer')[0]) 
        if matchcount == 0:
            print('no features matched spatial and attribute criteria')
        else:
            arcpy.CopyFeatures_management('tempLayer', (f'{selectFrom}_{rand}_{selType}'))
            display(f'{matchcount} Features selected and written to: {selectFrom}_{rand}_{selType}')
    except:
        display(f'ERROR Creating Selection: {selectFrom} {selType} {selector}')
        
def setCRS(envPath):
    try:
        test, count = 1, 0
        while test != 0:    
            dirList,layList = [], []
            arcpy.env.workspace = envPath
            arcpy.env.overwriteOutput = True
            print(f'ArcPy Workspace: {arcpy.env.workspace}')
            os.chdir(arcpy.env.workspace)
            print(f'Subdirectories')
            for i, item in enumerate(os.scandir(envPath)):
                if item.is_dir():
                    dirList.append(item.path)
                    print(f'{i} - {item.path}')
            ind = int(input(f'Change to Sub-Directory?<ENTER for none> ') or 999)
            if ind != 999:
                arcpy.env.workspace = dirList[ind]
                print(f'ArcPy Workspace: {arcpy.env.workspace}')
            for i, item in enumerate(arcpy.ListFeatureClasses()):
                layList.append(item)
                print(f'{i} - {item}')
            ind = int(input(f'TARGET LAYER TO DEFINE PROJECTION: ') or 999)
            if ind != 999:
                CRS = arcpy.SpatialReference(4326)
                arcpy.DefineProjection_management(layList[ind],CRS)
                print(f'Successfully Defined Projection for {layList[ind]}')
                count += 1
            test = int(input(f'PRESS 1 TO CONTINUE? ') or 0)
        print(f'Set CRS for {count} layers')
    except:
        print(f'ERROR Defining Projection')

## ArcPro Mapping Functions 2021


In [34]:
def setDir():
    # os.chdir('Y:\\PROJ-2021\\TEMPLATES')
    curDir = os.path.abspath(os.getcwd())
    listDir = os.listdir()
    print(f'Current Directory: {curDir}')
    print(f'Directory Contents:')
    for i, item in enumerate(listDir):
        print(f'{i} - {item}')
    ind = int(input(f'Change to Sub-Directory?<ENTER for none> ') or 999)
    if ind != 999:
        os.chdir(f'{curDir}\\{listDir[ind]}')
        curDir = os.path.abspath(os.getcwd())
        listDir = os.listdir()
        print(f'Current Directory: {curDir}')
        print(f'Directory Contents:')
        for i, item in enumerate(listDir):
            print(f'{i} - {item}')
    pathName = input(f'Enter Path to change Directory?<ENTER for none> ') or 'NO'
    if pathName != 'NO':
        os.chdir(pathName)
        curDir = os.path.abspath(os.getcwd())
    return curDir

def loadProj():
    curDir = os.path.abspath(os.getcwd())
    listDir = os.listdir()
    print(f'ArcPro Projects in Directory: ')
    for i, item in enumerate(listDir):
        print(f'{i} - {item}')
    ind = int(input(f'Project Index Number? ') or 999)
    aprx = arcpy.mp.ArcGISProject(f'{curDir}\\{listDir[ind]}')
    print(f'PROJECT LOADED: {listDir[ind]} - {aprx.filePath}')
    return aprx

def getMap(aprx):
    maps = aprx.listMaps('*')
    for i, item in enumerate(maps):
        print(f'{i} - {item.name}')
    ind = int(input(f'Choose Map Index? ') or 999)
    display(f'Active Map is: {maps[ind].name}')
    return maps[ind]

def getLayer(m):
    lyrs = m.listLayers()
    for i, item in enumerate(lyrs):
        print(f'{i} - {item.name} - vis: {item.visible} - trans: {item.transparency}')
    ind = int(input(f'Choose Layer Index? ') or 999)
    display(f'Layer: {lyrs[ind].name}')
    return lyrs[ind]

def getLayout(aprx):
    lyt = aprx.listLayouts()
    for i, item in enumerate(lyt):
        print(f'{i} - {item.name}')
    ind = int(input(f'Choose Layout Index? ') or 999)
    display(f'Layer: {lyt[ind].name}')
    return lyt[ind]

def getElement(lyt):
    el = lyt.listElements()
    for i, item in enumerate(el):
        print(f'{i} - {item.name}')
    ind = int(input(f'Choose Element Index? ') or 999)
    display(f'Layer: {el[ind].name}')
    return el[ind]

def addLyr(m):
    #try:
        lyrDir = 'Y:\\LAYERS\\'
        lyrs = os.listdir(lyrDir)
        for i, item in enumerate(lyrs):
            print(f'{i} - {item}')
        ind = int(input(f'Choose Layer Index? ') or 999)
        #dirPath = 'Y:\\LAYERS\\'
        layerPath = (f'{lyrDir}{lyrs[ind]}')
        lyr = arcpy.mp.LayerFile(layerPath)
        addLyr = m.addLayer(lyr, 'TOP')
        print(f'{lyr} added to {m.name}')


In [28]:
# terminal bash list current directory contents
%ls

 Volume in drive Y is EnvShare
 Volume Serial Number is 5CC6-B96D

 Directory of Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016

07/07/2021  04:46 PM    <DIR>          .
07/07/2021  04:46 PM    <DIR>          ..
07/08/2021  10:19 AM    <DIR>          AUGUSTA
07/07/2021  03:44 PM    <DIR>          FLOOD_2016
07/07/2021  04:56 PM    <DIR>          KETCHUM
07/07/2021  04:57 PM    <DIR>          KZOO_RIVER
07/08/2021  10:21 AM    <DIR>          MARL_DOWN_LEFT
07/07/2021  05:01 PM    <DIR>          MARL_DOWN_RIGHT
07/07/2021  05:01 PM    <DIR>          MORROW_DELTA
07/07/2021  05:01 PM    <DIR>          Mud_Correl_right_side
07/08/2021  10:01 AM    <DIR>          O_DRIVE_DOWN
07/08/2021  10:02 AM    <DIR>          O_DRIVE_DOWN_2
07/08/2021  10:02 AM    <DIR>          O_DRIVE_UP
07/08/2021  10:03 AM    <DIR>          PINE_TRANS_2
07/08/2021  10:03 AM    <DIR>          PINE_TRANSPLANTS
07/08/2021  10:03 AM    <DIR>          RT_SIDE
07/08/2021  10:04 AM    <DIR>          SHORTS_BRIDGE_LEFT
07/08/2

In [39]:
arcpy.env.workspace = 'Y:\\DATA_2017\\RICE_DATA_2021\\RICE_DATA_2016\\MARL_DOWN_LEFT'
arcpy.env.workspace

'Y:\\DATA_2017\\RICE_DATA_2021\\RICE_DATA_2016\\MARL_DOWN_LEFT'

In [40]:
listLayers = listItems(arcpy.env.workspace)

Feature Classes/Shapefile in Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_LEFT: 
0 - Line_gen.shp
1 - Wild_Ri2.shp
2 - Wild_Ri3.shp
3 - Wild_Ri4.shp
4 - Wild_Ric.shp


In [41]:
# Created function for the define CRS operation, workflow below
path = 'Y:\\DATA_2017\\RICE_DATA_2021\\RICE_DATA_2016'
results = setCRS(path)

ArcPy Workspace: Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016
Subdirectories
0 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\AUGUSTA
1 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\FLOOD_2016
2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\KETCHUM
3 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\KZOO_RIVER
4 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_LEFT
5 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_RIGHT
6 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MORROW_DELTA
7 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\Mud_Correl_right_side
8 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_DOWN
9 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_DOWN_2
10 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_UP
11 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\PINE_TRANSPLANTS
12 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\PINE_TRANS_2
13 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\RT_SIDE
14 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\SHORTS_BRIDGE_LEFT
15 - Y:\DATA_20